In [1]:

import codecs
from numpy import *
import matplotlib.pyplot as plt

def plot_cluster(data_mat, cluster_assment, centroid):
	"""
	@brief      plot cluster and centroid
	@param      data_mat        The data matrix
	@param      cluster_assment  The cluste assment
	@param      centroid        The centroid
	@return     
	"""
	plt.figure(figsize=(15, 6), dpi=80)
	plt.subplot(121)
	plt.plot(data_mat[:, 0], data_mat[:, 1], 'o')
	plt.title("source data", fontsize=15)
	plt.subplot(122)
	k = shape(centroid)[0]
	colors = [plt.cm.Spectral(each) for each in linspace(0, 1, k)]
	for i, col in zip(range(k), colors):
	    per_data_set = data_mat[nonzero(cluster_assment[:,0].A == i)[0]]
	    plt.plot(per_data_set[:, 0], per_data_set[:, 1], 'o', markerfacecolor=tuple(col),
	             markeredgecolor='k', markersize=10)
	for i in range(k):
		plt.plot(centroid[:,0], centroid[:,1], '+', color = 'k', markersize=18)
	plt.title("bi_KMeans Cluster, k = 3", fontsize=15)
	plt.show()


In [95]:
def fileInput(filename, columnNum):
    """
    @brief      read data from file
    @param      filename
    @param      columnNum: the range of columns to be read, start from 1
    @return     points_mat: [[position1], [position2], [position3], ... [positionN]]
    """   
    points_set = []

    with codecs.open(filename) as f:
        for line in f.readlines():
            linetmp = line.strip().split()[0:columnNum]
            #! map return a map object, 
            #! so we need to convert it to list or tuple by list(map()) or tuple(map())
            flisttmp = list(map(float, linetmp))
            points_set.append(flisttmp) 
    
    points_mat = mat(points_set)

    return points_mat


def centroid_gen(points_mat, k):
    """
    @brief      generate k centroids
    @param      points_set
    @param      k:          the number of centroids
    @return     centroids:  [[position1], [position2], [position3], ... [positionN]]
    """
    columnNum = shape(points_mat)[1]
    centroids = mat(zeros((k, columnNum)))

    for j in range(columnNum):
        mintmp = min(points_mat[:, j])
        maxtmp = max(points_mat[:, j])
        rangetmp = float(maxtmp - mintmp)
        centroids[:,j] = mat(mintmp + rangetmp * random.rand(k, 1))

    return centroids

def centroid_gen_fromMat(centroid_mat, k):
    """
    @brief      generate k centroids from the given points set
    @param      epsilon:            the range of the random centroids #! global variable
    @param      centroid_mat:       the points set that is used to generate random centroids
    @param      k:                  the number of centroids
    @return     centroids:          [[position1], [position2], [position3], ... [positionN]]
    """

    centroids = mat(zeros((k, shape(centroid_mat)[1])))
    #* in box, I'm lazy to write the algorithm in sphere
    for i in range(k):
        for j in range(shape(centroid_mat)[1]):
            mintmp = centroid_mat[i,j] - epsilon
            maxtmp = centroid_mat[i,j] + epsilon
            rangetmp = float(maxtmp - mintmp)
            centroids[i,j] = mat(mintmp + rangetmp * random.rand(1))
    
    return centroids



def distance_Euclidean(vecA, vecB):
    """
    @brief      calculate the Euclidean distance between two vectors
    @param      vecA
    @param      vecB
    @return     the Euclidean distance
    """
    return sqrt(sum(power(vecA - vecB, 2)))

def k_means(points_mat, k, distance_calculation, generate_method):
    """
    @brief      kMeans algorithm
    @param      points_mat:             The data matrix
    @param      k:                      num of cluster
    @param      distance_calculation:   The distance funtion
    @return     centroids
    @return     cluster_attribute:      [[cluster_index, distance**2], ...]
    """

    pointsNum = shape(points_mat)[0]

    cluster_attribute = mat(zeros((pointsNum, 2)))  # cluster index, distance

    #* Initialize the centroids

    cluster_num_change = True

    while cluster_num_change:
        cluster_num_change = False

        if generate_method == "centroid_gen":
            centroids = centroid_gen(points_mat, k)
        elif generate_method == "centroid_gen_fromMat":
            centroids = centroid_gen_fromMat(centroid_mat, k)


        cluster_changed = True

        #* loop until the cluster is not changed
        while cluster_changed:
            cluster_changed = False

            #* assign each point to the nearest centroid
            for i in range(pointsNum):
                cluster_index = None
                min_dist = inf
                for j in range(k):
                    dist_tmp = eval(distance_calculation)(points_mat[i, :], centroids[j, :])
                    if dist_tmp < min_dist:
                        min_dist = dist_tmp
                        cluster_index = j
                
                #* judge if the cluster is changed
                if cluster_attribute[i, 0] != cluster_index:
                    cluster_changed = True

                cluster_attribute[i, :] = cluster_index, min_dist**2

            #!cluster_attribute[:,0].T.tolist()-> [[]]
            if len(set(cluster_attribute[:,0].T.tolist()[0])) != k:
                cluster_num_change = True
                #print(f"The number of mapped clusters ({len(set(cluster_attribute[:,0].T.tolist()[0]))}) is less than {k}, re-initialize the centroids")
                continue

            #! ONLY if all clusters are mapped to points
            #! than update the centroids, to avoid `mean` in empty array
            #* update the centroids, in the new clusters
            for j in range(k):
                pointstmp = points_mat[nonzero(cluster_attribute[:, 0] == j)[0]]
                centroids[j, :] = mean(pointstmp, axis = 0)

            #* check if the total number of clusters is less than k



    return centroids, cluster_attribute

#! there is the case
#! that all the points are in one cluster
def bi_kmeans(points_mat, k, distance_calculation):
    """
    @brief      bi-KMeans algorithm
    @param      points_mat:             The data matrix
    @param      k:                      num of cluster
    @param      distance_calculation:   The distance funtion
    @return     centroids_bi:           [[position1], [position2], [position3], ... [positionN]]
    @return     cluster_attribute:      [[cluster_index, distance**2], ...]
    """

    centroids_bi = []
    pointsNum = shape(points_mat)[0]
    cluster_attribute = mat(zeros((pointsNum, 2)))  # cluster index, distance

    #* Initialize the centroids
    #! at first, only one centroid
    #! here the output of mean is a matrix, with two shells [[x,x]]
    centroids_bi.append(mean(points_mat, axis = 0).tolist()[0])

    #* Initialize the cluster attribute
    for i in range(pointsNum):
        cluster_attribute[i,1] = eval(distance_calculation)(centroids_bi[0], points_mat[i, :]) ** 2

    while len(centroids_bi) < k:
        lowest_sse = inf
        for i in range(len(centroids_bi)):
            points_mat_tmp = points_mat[nonzero(cluster_attribute[:, 0] == i)[0],:]
            #* there are two clusters, index0 and index1
            centroids_bi_tmp, cluster_attribute_tmp = k_means(points_mat_tmp, 2, distance_calculation, "centroid_gen")
            sse_split = sum(cluster_attribute_tmp[:, 1])
            #! only calculate in the new cluster
            sse_nonsplit = sum(cluster_attribute[nonzero(cluster_attribute[:, 0] != i)[0], 1])

            #* find the BEST split in all cluster and ONLY choose one in each loop
            if sse_split + sse_nonsplit < lowest_sse:
                best_split_index = i
                lowest_sse = sse_split + sse_nonsplit
                best_cluster_attribute = cluster_attribute_tmp.copy()
                best_centroids_bi = centroids_bi_tmp.copy()

        #! for the split of cluster i
        #! i -> index 0, and len -> index 1 
        
        best_cluster_attribute[nonzero(best_cluster_attribute[:,0]==1)[0],0] = len(centroids_bi)
        best_cluster_attribute[nonzero(best_cluster_attribute[:,0]==0)[0],0] = best_split_index
        
        cluster_attribute[nonzero(cluster_attribute[:,0]==best_split_index)[0],:] = best_cluster_attribute

        centroids_bi[best_split_index] = best_centroids_bi[0,:].tolist()[0]
        centroids_bi.append(best_centroids_bi[1,:].tolist()[0])

    
    centroids_bi = mat(centroids_bi)
    
    return centroids_bi, cluster_attribute

def bi_kmeansLoop(points_mat, k, loopNum, distance_calculation):

    #* to view centroids as the set to be clustered
    centroid_mat = zeros((k*loopNum, shape(points_mat)[1]))

    for i in range(loopNum):
        centroids_bi, cluster_attribute = bi_kmeans(points_mat, k, distance_calculation)
        centroid_mat[i*k:(i+1)*k, :] = centroids_bi.copy()

    return centroid_mat


                


distance_calculation = "distance_Euclidean"
filename = "attractorsX.txt"
columnNum = 3
k = 14

points_mat = fileInput(filename, columnNum)

#centroids_bi, cluster_attribute = bi_kmeans(points_mat, k, distance_calculation)
centroid_mat = bi_kmeansLoop(points_mat, k, 20, distance_calculation)


In [96]:
Bohr2Angstrom = 0.529177249 

for i in range(shape(centroid_mat)[0]):
    print(f"Bq \t {centroid_mat[i,0]*Bohr2Angstrom} \t {centroid_mat[i,1]*Bohr2Angstrom} \t {centroid_mat[i,2]*Bohr2Angstrom} \t L")

Bq 	 0.0015028633871599962 	 -1.0583544979994453e-05 	 0.11198448943337998 	 L
Bq 	 -2.461379777515333 	 -2.36779830887801 	 -1.048829307518 	 L
Bq 	 2.01857836694544 	 -1.1710692520369999 	 1.6192612148500396 	 L
Bq 	 -1.9740498653662528 	 1.2342053896151894 	 1.5908743832895162 	 L
Bq 	 0.26484262957952 	 2.1887363697158877 	 -0.8750855997083281 	 L
Bq 	 -1.842682671656501 	 -1.0163406467080616 	 1.7847469352966505 	 L
Bq 	 0.0670855637798933 	 -2.1623899280325647 	 -0.8298839846684133 	 L
Bq 	 -1.9122507823038695 	 0.07061693995488666 	 -1.2571805018426068 	 L
Bq 	 -2.1718069311658796 	 2.465161630921519 	 -0.9934138660027199 	 L
Bq 	 0.0929823223965111 	 -0.001187708936644457 	 2.852164240457969 	 L
Bq 	 2.0113603892690803 	 1.1608032134064 	 1.48468085688436 	 L
Bq 	 1.8963384224264397 	 0.004227067865011999 	 -1.332751951987464 	 L
Bq 	 2.7038464411082486 	 -2.541424304148516 	 -1.0582463106513156 	 L
Bq 	 2.7038464411082486 	 2.541403137058555 	 -1.0558920598679864 	 L
Bq 	 -1.9

In [97]:
centroids_k, cluster_attribute = bi_kmeans(centroid_mat, k, distance_calculation)

centroid_mat = centroids_k

epsilon = 1.0   #* the radius of the random points, in Bohr

centroids, cluster_attribute = k_means(points_mat, k, distance_calculation, "centroid_gen_fromMat")



In [98]:
for i in range(shape(centroids)[0]):
    print(f"He \t {centroids[i,0]*Bohr2Angstrom} \t {centroids[i,1]*Bohr2Angstrom} \t {centroids[i,2]*Bohr2Angstrom} \t H")

He 	 0.016940022094987982 	 -0.0021294092499760144 	 2.84980998967464 	 H
He 	 -2.6930380545948958 	 2.551055565270204 	 -1.048829307518 	 H
He 	 1.8963384224264397 	 0.004227067865011999 	 -1.332751951987464 	 H
He 	 2.0464219098636565 	 -1.2642432541925932 	 1.5573263096270797 	 H
He 	 0.0015028633871599962 	 -1.0583544979994453e-05 	 0.11198448943337998 	 H
He 	 1.9895667534462642 	 1.0798842507679811 	 1.5432008049271067 	 H
He 	 2.7038464411082486 	 -2.541424304148516 	 -1.0582463106513156 	 H
He 	 -1.9191777124932794 	 0.008953679053079996 	 -1.25093268245608 	 H
He 	 -1.9501451651047594 	 1.1384507664086396 	 1.65022866746152 	 H
He 	 -2.6930380545948958 	 -2.5489579066551675 	 -1.055185784632988 	 H
He 	 2.7038464411082486 	 2.541403137058555 	 -1.0558920598679864 	 H
He 	 0.0026379485862649907 	 2.185028424732145 	 -0.8501893976746249 	 H
He 	 -1.0583544980006544e-05 	 -2.1654504540508923 	 -0.832709085608408 	 H
He 	 -1.9468854332509196 	 -1.1710692520370003 	 1.6600078630230

In [14]:
attractors = list(set(cluster_attribute[:,0].T.tolist()[0]))

for i in range(len(attractors)):
    temp = nonzero(cluster_attribute[:,0] == attractors[i])[0].tolist()
    #* transform the index to the real index
    temp = [x+1 for x in temp]
    print(f"cluster: {attractors[i]}, len: {len(temp)}, attractors: {temp}")




cluster: 0.0, len: 17, attractors: [4, 43, 50, 51, 64, 67, 73, 76, 89, 93, 102, 112, 114, 116, 124, 126, 131]
cluster: 1.0, len: 13, attractors: [44, 68, 75, 78, 80, 84, 92, 101, 141, 144, 147, 149, 156]
cluster: 2.0, len: 9, attractors: [5, 7, 11, 20, 22, 23, 24, 25, 72]
cluster: 3.0, len: 10, attractors: [3, 8, 10, 12, 14, 17, 18, 21, 49, 57]
cluster: 4.0, len: 11, attractors: [1, 6, 9, 13, 15, 16, 19, 28, 48, 58, 153]
cluster: 5.0, len: 10, attractors: [26, 29, 30, 35, 36, 39, 53, 54, 99, 108]
cluster: 6.0, len: 12, attractors: [106, 113, 115, 117, 118, 123, 125, 127, 128, 138, 140, 145]
cluster: 7.0, len: 13, attractors: [96, 110, 111, 120, 121, 130, 132, 134, 135, 137, 139, 142, 143]
cluster: 8.0, len: 12, attractors: [27, 37, 38, 40, 41, 52, 82, 104, 105, 122, 154, 155]
cluster: 9.0, len: 12, attractors: [46, 61, 65, 69, 74, 79, 81, 85, 87, 100, 148, 150]
cluster: 10.0, len: 11, attractors: [2, 42, 47, 56, 62, 71, 77, 83, 95, 103, 151]
cluster: 11.0, len: 14, attractors: [31, 32,

In [148]:
attractors = list(set(cluster_attribute[:,0].T.tolist()[0]))

for i in range(len(attractors)):
    temp = nonzero(cluster_attribute[:,0] == attractors[i])[0].tolist()
    #* transform the index to the real index
    temp = [x+1+14 for x in temp]
    print(f"cluster: {attractors[i]}, len: {len(temp)}, attractors in gjf: {temp}")

cluster: 0.0, len: 10, attractors in gjf: [106, 123, 136, 143, 147, 150, 155, 159, 160, 167]
cluster: 1.0, len: 14, attractors in gjf: [125, 130, 133, 134, 135, 137, 139, 141, 142, 144, 145, 149, 154, 158]
cluster: 2.0, len: 13, attractors in gjf: [18, 57, 62, 63, 75, 79, 81, 83, 98, 100, 113, 114, 128]
cluster: 3.0, len: 14, attractors in gjf: [15, 20, 23, 25, 27, 29, 30, 33, 34, 37, 42, 64, 72, 166]
cluster: 4.0, len: 9, attractors in gjf: [107, 124, 126, 138, 146, 148, 151, 156, 157]
cluster: 5.0, len: 17, attractors in gjf: [16, 56, 60, 69, 74, 80, 86, 90, 101, 115, 127, 129, 131, 132, 152, 153, 165]
cluster: 6.0, len: 8, attractors in gjf: [19, 21, 22, 32, 36, 38, 39, 87]
cluster: 7.0, len: 14, attractors in gjf: [45, 46, 47, 48, 58, 66, 73, 77, 99, 102, 103, 108, 109, 117]
cluster: 8.0, len: 12, attractors in gjf: [41, 51, 52, 54, 55, 67, 93, 118, 119, 140, 168, 169]
cluster: 9.0, len: 13, attractors in gjf: [59, 76, 85, 89, 92, 94, 96, 104, 110, 121, 161, 163, 170]
cluster: 10.0

In [119]:
# read the intergration of basin

basin_filename = "basinIntX.txt"

basin_int_list = []

basin_int_pos = zeros((shape(points_mat)[0],2))   #the positive basin [index, value]
basin_int_neg = zeros((shape(points_mat)[0],2))   #the negative basin [index, value]

with codecs.open(basin_filename) as f:
    #* jump the first line
    f.readline()
    for iline in range(shape(points_mat)[0]):
        line = f.readline()
        basinIntX = line.strip().split()[1]
        basin_int_list.append(float(basinIntX))

basin_int_list = mat(basin_int_list)

pos_index = nonzero(basin_int_list > 0)[1].tolist()
neg_index = nonzero(basin_int_list < 0)[1].tolist()

for i in range(len(pos_index)):
    basin_int_pos[pos_index[i],0] = pos_index[i] + 1
    basin_int_pos[pos_index[i],1] = basin_int_list[0,pos_index[i]]

for i in range(len(neg_index)):
    basin_int_neg[neg_index[i],0] = neg_index[i] + 1
    basin_int_neg[neg_index[i],1] = basin_int_list[0,neg_index[i]]

In [149]:
print(f"sum of positive basin: {sum(basin_int_pos[:,1])}")
print(f"sum of negative basin: {sum(basin_int_neg[:,1])}")
print(f"sum of basin: {sum(basin_int_neg[:,1]) + sum(basin_int_pos[:,1])}")

sum of positive basin: 1125.671977153
sum of negative basin: -1125.6845046383
sum of basin: -0.012527485300097396


In [136]:
Vindex = [31, 32, 33, 34, 44, 52, 59, 63, 85, 88, 89, 94, 95, 103]

Vpos = []
Vneg = []

for i in Vindex:
    Vpos.append(basin_int_pos[i-1,1])
    Vneg.append(basin_int_neg[i-1,1])

In [146]:
sum(Vpos)

145.6673503769

In [147]:
sum(Vneg)

-117.7158977958